In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from nltk import FreqDist
import gensim
from sklearn.metrics import f1_score

Using Theano backend.
C:\Users\shyam\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df = pd.read_csv('ner_dataset.csv')
df

Sentence #           Word  POS    Tag
0            Sentence: 1      Thousands  NNS      O
1                    NaN             of   IN      O
2                    NaN  demonstrators  NNS      O
3                    NaN           have  VBP      O
4                    NaN        marched  VBN      O
5                    NaN        through   IN      O
6                    NaN         London  NNP  B-geo
7                    NaN             to   TO      O
8                    NaN        protest   VB      O
9                    NaN            the   DT      O
10                   NaN            war   NN      O
11                   NaN             in   IN      O
12                   NaN           Iraq  NNP  B-geo
13                   NaN            and   CC      O
14                   NaN         demand   VB      O
15                   NaN            the   DT      O
16                   NaN     withdrawal   NN      O
17                   NaN             of   IN      O
18                   NaN        British   JJ  B-gpe
19                   NaN         troops  NNS      O
20                   NaN           from   IN      O
21                   NaN           that   DT      O
22                   NaN        country   NN      O
23                   NaN              .    .      O
24           Sentence: 2       Families  NNS      O
25                   NaN             of   IN      O
26                   NaN       soldiers  NNS      O
27                   NaN         killed  VBN      O
28                   NaN             in   IN      O
29                   NaN            the   DT      O
...                  ...            ...  ...    ...
1048545  Sentence: 47957            Two   CD      O
1048546              NaN           more  JJR      O
1048547              NaN         landed  VBD      O
1048548              NaN             in   IN      O
1048549              NaN         fields  NNS      O
1048550              NaN      belonging  VBG      O
1048551              NaN             to   TO      O
1048552              NaN              a   DT      O
1048553              NaN         nearby   JJ      O
1048554              NaN        village   NN      O
1048555              NaN              .    .      O
1048556  Sentence: 47958           They  PRP      O
1048557              NaN            say  VBP      O
1048558              NaN            not   RB      O
1048559              NaN            all   DT      O
1048560              NaN             of   IN      O
1048561              NaN            the   DT      O
1048562              NaN        rockets  NNS      O
1048563              NaN       exploded  VBD      O
1048564              NaN           upon   IN      O
1048565              NaN         impact   NN      O
1048566              NaN              .    .      O
1048567  Sentence: 47959         Indian   JJ  B-gpe
1048568              NaN         forces  NNS      O
1048569              NaN           said  VBD      O
1048570              NaN           they  PRP      O
1048571              NaN      responded  VBD      O
1048572              NaN             to   TO      O
1048573              NaN            the   DT      O
1048574              NaN         attack   NN      O

[1048575 rows x 4 columns]

In [26]:
txt = ""
txt += df.iloc[0,df.columns.get_loc('Word')]

for i in xrange(1, df.shape[0]-1):
        txt += " "
        txt += df.iloc[i,df.columns.get_loc('Word')]

with open("Story.txt", "w") as text_file:
    text_file.write(txt)

In [4]:
no_of_datapoints = 1000
data = open("Story.txt", "r").read()
charactersList = list(set(data))
print len(data)
print len(charactersList)

char_to_index = {char:index for index,char in enumerate(charactersList)}

6053792
99


In [5]:
characterOneHotList = []

for i in xrange(0, no_of_datapoints):
    chars = list(df.iloc[i, df.columns.get_loc('Word')])
    
    char_oneHot = []
    
    for j in xrange(0, len(chars)):
        char_oneHot.append(char_to_index[chars[j]])
    
    characterOneHotList.append(char_oneHot)

print len(characterOneHotList)
print characterOneHotList[0]

1000
[14, 18, 46, 96, 47, 91, 69, 17, 47]


In [6]:
characterOneHotListPadded = pad_sequences(characterOneHotList)

print characterOneHotListPadded.shape

(1000, 17)


In [7]:
sentence_character_list = []
sentence_character = [characterOneHotListPadded[0]]

for i in xrange(1, no_of_datapoints):
    if df['Sentence #'].isnull()[i] == True:
        sentence_character.append(characterOneHotListPadded[i])
    else: 
        sentence_character_list.append(sentence_character)
        sentence_character = []
        sentence_character.append(characterOneHotListPadded[i])

sentence_character_list = np.array(sentence_character_list)

print sentence_character_list.shape

(42,)


In [8]:
print len(sentence_character_list[1])
sentence_character_list_padded = pad_sequences(sentence_character_list)
print sentence_character_list_padded.shape
print sentence_character_list_padded[0]

30
(42, 40, 17)
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 14 18 46 96 47 91 69 17 47]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 46 66]
 [ 0  0  0  

In [9]:
freqdist = FreqDist(df.iloc[:no_of_datapoints, df.columns.get_loc('Word')])

vocab = []
for key in freqdist:
    if key.isalpha():
        vocab.append(key)

vocab.append('UNKNOWN')
print len(vocab)

477


In [10]:
sentences = []
x = vocab.index('UNKNOWN')
wordToIndex = [vocab.index(df.iloc[0, df.columns.get_loc('Word')])]
        
for i in xrange(1, no_of_datapoints):
    word = df.iloc[i, df.columns.get_loc('Word')]

    if df['Sentence #'].isnull()[i] == True:
        if word in vocab:
            wordToIndex.append(vocab.index(word))
        else:
            wordToIndex.append(x)
    else: 
        sentences.append(wordToIndex)
        wordToIndex = []
        if word in vocab:
            wordToIndex.append(vocab.index(word))
        else:
            wordToIndex.append(x)

print sentences

[[429, 416, 404, 216, 351, 122, 393, 9, 2, 360, 435, 92, 197, 213, 317, 360, 270, 416, 107, 61, 60, 200, 298, 476], [457, 416, 247, 377, 92, 360, 345, 151, 360, 386, 230, 414, 474, 439, 184, 38, 454, 476, 333, 321, 115, 427, 476, 213, 476, 339, 360, 399, 476, 476], [447, 351, 60, 360, 64, 416, 108, 9, 112, 456, 92, 177, 285, 476], [90, 169, 360, 181, 416, 67, 455, 476, 325, 421, 335, 332, 434, 476, 476], [233, 2, 305, 413, 360, 17, 416, 360, 300, 310, 416, 445, 476, 446, 365, 94, 92, 360, 106, 40, 395, 131, 416, 70, 476], [233, 344, 331, 98, 121, 445, 476, 405, 92, 360, 197, 345, 213, 360, 14, 388, 416, 476, 107, 61, 92, 200, 298, 476], [233, 393, 382, 241, 42, 416, 476, 423, 69, 92, 462, 295, 476, 271, 137, 476, 243, 476, 213, 172, 476], [233, 401, 211, 43, 244, 331, 9, 80, 25, 144, 416, 3, 92, 402, 76, 413, 100, 9, 273, 9, 188, 476, 453, 416, 476, 459, 219, 476], [188, 18, 346, 442, 342, 416, 360, 219, 91, 455, 125, 280, 284, 103, 476], [12, 278, 215, 139, 155, 9, 263, 29, 9, 433, 30

In [ ]:
import pickle

with open('sentenceIndex.txt', 'wb') as fp:
    pickle.dump(sentences, fp)

In [ ]:
import pickle

with open ('sentenceIndex.txt', 'rb') as fp:
    sentences = pickle.load(fp)

print sentences
print len(sentences)

In [11]:
sentencesPadded = pad_sequences(sentences)
print sentencesPadded.shape

(42, 40)


In [ ]:
EMBEDDING_DIM = 300

wordPretrainedModel = gensim.models.Word2Vec.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)  

wordEmbeddingPretrainedMatrix = np.zeros((len(vocab), EMBEDDING_DIM))

for i in xrange(0, len(vocab)):
    if word in 
    wordEmbeddingPretrainedMatrix[i] = (model.wv[vocab[i]])

print wordEmbeddingPretrainedMatrix.shape

In [12]:
EntityOneHot = pd.get_dummies(df.iloc[:no_of_datapoints,df.columns.get_loc('Tag')])

labels = np.empty((EntityOneHot.shape), dtype = int)

for i in xrange(0,EntityOneHot.shape[1]):
    numbers = np.array(EntityOneHot.iloc[:,i])
    labels[:,i] = numbers
              
print len(labels)
print labels

labels_sequence_list = []
labels_sequence = [labels[0]]
        
for i in xrange(1, no_of_datapoints):
    if df['Sentence #'].isnull()[i] == True:
            labels_sequence.append(labels[i])
    else: 
        labels_sequence_list.append(labels_sequence)
        labels_sequence = []
        labels_sequence.append(labels[i])

labels_sequence_list = np.array(labels_sequence_list)
print labels_sequence_list.shape

1000
[[0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]]
(42,)


In [13]:
labels_sequence_list_padded = pad_sequences(labels_sequence_list)
print labels_sequence_list_padded.shape

(42, 40, 11)


In [15]:
print len(charactersList)

99


In [70]:
EMBEDDING_DIM = 300
main_input = Input(shape = (sentence_character_list_padded.shape[1], sentence_character_list_padded.shape[2],),
                   dtype = 'float32', name = 'main_input')
print main_input._keras_shape

character_embedding_intermediate = Embedding(input_dim = len(charactersList), output_dim = 30, 
                                input_length = (sentence_character_list_padded.shape[1], sentence_character_list_padded.shape[2])) (main_input)
print character_embedding_intermediate._keras_shape

LSTM_character_forward = TimeDistributed(LSTM(64, return_sequences = False, go_backwards = False))(character_embedding_intermediate)
LSTM_character_backward = TimeDistributed(LSTM(64, return_sequences = False, go_backwards = True))(character_embedding_intermediate)
character_embedding = keras.layers.concatenate([LSTM_character_forward, LSTM_character_backward])
print character_embedding._keras_shape

auxillary_input = Input(shape = (sentencesPadded.shape[1],), dtype = 'float32', name = 'second_input')
word_pretrained_embedding = Embedding(input_dim = len(vocab), output_dim = EMBEDDING_DIM, 
                                      input_length = sentencesPadded.shape[1])(auxillary_input)
#                                      weights = [wordEmbeddingPretrainedMatrix], trainable = True)  
print word_pretrained_embedding._keras_shape  

word_embedding = keras.layers.concatenate([word_pretrained_embedding, character_embedding])
print word_embedding._keras_shape

LSTM_out_forward = LSTM(128, go_backwards = False, return_sequences = True)(word_embedding)
LSTM_out_backward = LSTM(128, go_backwards = True, return_sequences = True)(word_embedding)
LSTM_out = keras.layers.concatenate([LSTM_out_forward, LSTM_out_backward])
print LSTM_out._keras_shape

dense_out_1 = Dense(256, activation = 'relu')(LSTM_out)
print dense_out_1._keras_shape

dropout_1 = Dropout(0.4)(dense_out_1)

main_output = Dense(labels.shape[1], activation = 'softmax', name = 'main_output')(dropout_1)
print main_output._keras_shape

model = Model(inputs = [main_input, auxillary_input], outputs = [main_output])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

(None, 40, 17)
(None, 40, 17, 30)
(None, 40, 128)
(None, 40, 300)
(None, 40, 428)
(None, 40, 256)
(None, 40, 256)
(None, 40, 11)


In [71]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 40, 17)        0                                            
____________________________________________________________________________________________________
embedding_9 (Embedding)          (None, 40, 17, 30)    2970        main_input[0][0]                 
____________________________________________________________________________________________________
second_input (InputLayer)        (None, 40)            0                                            
____________________________________________________________________________________________________
time_distributed_9 (TimeDistribu (None, 40, 64)        24320       embedding_9[0][0]                
___________________________________________________________________________________________

In [74]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import f1_score

class Metrics(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict([self.validation_data[0], self.validation_data[1]]))
        
        argmax = np.argmax(predict, axis = 2)
        
        predictions = np.zeros((predict.shape[0], predict.shape[1], predict.shape[2]), dtype = int)
        for i in xrange(0,argmax.shape[0]):
            for j in xrange(0, argmax.shape[1]):
                predictions[i][j][argmax[i][j]] = 1
                
        predictions = predictions.reshape((-1,predict.shape[2]))

        targ = self.validation_data[2]
        targ = targ.reshape((-1,predict.shape[2]))

        f1s = f1_score(targ, predictions, average = 'micro')
        print "f1_score = ", f1s
        return

In [73]:
metrics = Metrics()
x = 32
#model.fit([sentence_character_list_padded, sentencesPadded],[labels_sequence_list_padded], epochs = 3, batch_size = 1)
model.fit([sentence_character_list_padded[:x], sentencesPadded[:x]],[labels_sequence_list_padded[:x]], 
          validation_data = [[sentence_character_list_padded[x:], sentencesPadded[x:]],labels_sequence_list_padded[x:]],
          epochs = 3, batch_size = 1, callbacks = [metrics])

Train on 32 samples, validate on 10 samples
Epoch 1/3
32/32 [==============================] - 17s - loss: 0.7077 - acc: 0.4742 - val_loss: 0.3430 - val_acc: 0.5625
Epoch 2/3
32/32 [==============================] - 15s - loss: 0.4547 - acc: 0.4898 - val_loss: 0.3362 - val_acc: 0.5625
Epoch 3/3
32/32 [==============================] - 15s - loss: 0.4234 - acc: 0.4898 - val_loss: 0.3354 - val_acc: 0.5625
